In [2]:
from pathlib import Path
import xarray as xr
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# ---------- paths ----------
PROJECT_ROOT = Path(r"C:\Users\s572269\Downloads\river_plastic_predictor")
DATA_DIR = PROJECT_ROOT / "data"
RAW_PATH = DATA_DIR / "raw"
PROC_PATH = DATA_DIR / "processed"
PROC_PATH.mkdir(parents=True, exist_ok=True)

print("RAW_PATH:", RAW_PATH)
print("PROC_PATH:", PROC_PATH)

# ---------- load rivers ----------
rivers = pd.read_csv(RAW_PATH / "global_rivers_dataset.csv")
print(f"Total rivers: {len(rivers)}")

# Fix river coordinates - they appear to be in some projection, not degrees
# Let's try a different approach: normalize and approximate
print("\nAnalyzing river coordinate ranges:")
print(f"Min/Max mouth_lat: {rivers['mouth_lat'].min():.2f}, {rivers['mouth_lat'].max():.2f}")
print(f"Min/Max mouth_lon: {rivers['mouth_lon'].min():.2f}, {rivers['mouth_lon'].max():.2f}")

# Based on the previous analysis, let's try scaling by 1e5 instead
# This gives more reasonable degree values
rivers["mouth_lat_deg"] = rivers["mouth_lat"] / 1e5
rivers["mouth_lon_deg"] = rivers["mouth_lon"] / 1e5

print("\nAfter scaling by 100,000:")
print(rivers[['river_name', 'mouth_lat_deg', 'mouth_lon_deg']].head(10))
print(f"\nLat range: {rivers['mouth_lat_deg'].min():.2f} to {rivers['mouth_lat_deg'].max():.2f}")
print(f"Lon range: {rivers['mouth_lon_deg'].min():.2f} to {rivers['mouth_lon_deg'].max():.2f}")

# Clean up extreme values
rivers.loc[rivers['mouth_lat_deg'] > 90, 'mouth_lat_deg'] = 90
rivers.loc[rivers['mouth_lat_deg'] < -90, 'mouth_lat_deg'] = -90
rivers.loc[rivers['mouth_lon_deg'] > 180, 'mouth_lon_deg'] = 180
rivers.loc[rivers['mouth_lon_deg'] < -180, 'mouth_lon_deg'] = -180

# Create filtered dataset
rivers_small = rivers[["river_name", "main_country", "mouth_lat_deg", "mouth_lon_deg"]].copy()
rivers_small = rivers_small.rename(columns={"main_country": "country"}).dropna()

print(f"\nRivers after cleaning: {len(rivers_small)}")

# ---------- Find all HYCOM files ----------
# Find all files starting with uv3z_ in the raw directory
hycom_files = sorted(list(RAW_PATH.glob("uv3z_*.nc4")))
print(f"\nFound {len(hycom_files)} HYCOM files:")
for f in hycom_files:
    print(f"  {f.name}")

if len(hycom_files) == 0:
    raise FileNotFoundError("No HYCOM files found! Expected files like uv3z_08-31-2024.nc4")

# ---------- Check first file to understand structure ----------
first_file = hycom_files[0]
print(f"\nExamining first file: {first_file.name}")
ds_sample = xr.open_dataset(first_file)

print("Dataset variables:", list(ds_sample.data_vars))
print("Dataset coordinates:", list(ds_sample.coords))
print("Dataset dimensions:", dict(ds_sample.dims))

# Get HYCOM bounds from first file
hycom_lat_min = float(ds_sample.lat.min())
hycom_lat_max = float(ds_sample.lat.max())
hycom_lon_min = float(ds_sample.lon.min())
hycom_lon_max = float(ds_sample.lon.max())

print(f"\nHYCOM region bounds:")
print(f"  Latitude: {hycom_lat_min:.1f}° to {hycom_lat_max:.1f}°")
print(f"  Longitude: {hycom_lon_min:.1f}° to {hycom_lon_max:.1f}°")

# Check what time periods are covered
print(f"\nTime in first file: {ds_sample.time.values}")
ds_sample.close()

# ---------- Filter rivers in HYCOM region ----------
# Adjust longitudes to match HYCOM's 0-360 system if needed
def adjust_lon_for_hycom(lon):
    """Adjust longitude to match HYCOM coordinate system"""
    # HYCOM seems to use 0-360, but check from data
    if hycom_lon_min >= 0 and hycom_lon_max <= 360 and lon < 0:
        return lon + 360
    return lon

rivers_small["lon_adjusted"] = rivers_small["mouth_lon_deg"].apply(adjust_lon_for_hycom)

# Find rivers within HYCOM region
in_region_mask = (
    (rivers_small["mouth_lat_deg"] >= hycom_lat_min) &
    (rivers_small["mouth_lat_deg"] <= hycom_lat_max) &
    (rivers_small["lon_adjusted"] >= hycom_lon_min) &
    (rivers_small["lon_adjusted"] <= hycom_lon_max)
)

rivers_in_region = rivers_small[in_region_mask].copy()
print(f"\nRivers within HYCOM region: {len(rivers_in_region)}")

if len(rivers_in_region) == 0:
    print("No rivers found in HYCOM region. Finding nearest rivers...")
    
    # Calculate simple distance to region center
    region_center_lat = (hycom_lat_min + hycom_lat_max) / 2
    region_center_lon = (hycom_lon_min + hycom_lon_max) / 2
    
    rivers_small['dist_to_region'] = np.sqrt(
        (rivers_small['mouth_lat_deg'] - region_center_lat)**2 +
        ((rivers_small['lon_adjusted'] - region_center_lon) * np.cos(np.radians(region_center_lat)))**2
    )
    
    # Take closest 100 rivers
    rivers_in_region = rivers_small.nsmallest(100, 'dist_to_region')
    print(f"Selected {len(rivers_in_region)} closest rivers")

print("\nSample rivers to process:")
print(rivers_in_region[['river_name', 'country', 'mouth_lat_deg', 'mouth_lon_deg', 'lon_adjusted']].head(10))

# ---------- Process all HYCOM files ----------
all_results = []

for file_idx, hycom_file in enumerate(hycom_files):
    print(f"\n{'='*60}")
    print(f"Processing file {file_idx+1}/{len(hycom_files)}: {hycom_file.name}")
    print(f"{'='*60}")
    
    # Open the current file
    ds = xr.open_dataset(hycom_file)
    
    # Parse date from filename
    date_str = hycom_file.stem.replace("uv3z_", "")
    try:
        file_date = datetime.strptime(date_str, "%m-%d-%Y").date()
    except:
        file_date = datetime.strptime(date_str, "%Y%m%d").date() if len(date_str) == 8 else None
    
    print(f"File date: {file_date}")
    print(f"Data time: {ds.time.values}")
    
    # Use bottom velocities if available, otherwise surface
    if "water_u_bottom" in ds.data_vars and "water_v_bottom" in ds.data_vars:
        u_name, v_name = "water_u_bottom", "water_v_bottom"
        depth_type = "bottom"
    else:
        u_name, v_name = "water_u", "water_v"
        depth_type = "surface"
    
    print(f"Using {depth_type} velocity variables: {u_name}, {v_name}")
    
    # Apply scaling factors if they exist
    for var_name in [u_name, v_name]:
        var = ds[var_name]
        if 'scale_factor' in var.attrs:
            scale = var.attrs['scale_factor']
            offset = var.attrs.get('add_offset', 0.0)
            fill_value = var.attrs.get('_FillValue', np.nan)
            
            # Apply scaling
            if not np.isnan(fill_value):
                mask = var == fill_value
                ds[var_name] = var.where(~mask) * scale + offset
                ds[var_name] = ds[var_name].where(ds[var_name] != fill_value)
            else:
                ds[var_name] = var * scale + offset
    
    # Select first time step if multiple
    if "time" in ds.dims and len(ds.time) > 1:
        ds_single = ds.isel(time=0)
    else:
        ds_single = ds
    
    # Process each river
    daily_results = []
    
    for idx, river_row in rivers_in_region.iterrows():
        lat = river_row["mouth_lat_deg"]
        lon = river_row["lon_adjusted"]
        
        # Clamp to HYCOM bounds
        lat_clamped = np.clip(lat, hycom_lat_min, hycom_lat_max)
        lon_clamped = np.clip(lon, hycom_lon_min, hycom_lon_max)
        
        try:
            # Try nearest neighbor interpolation
            pt = ds_single.sel(
                lat=lat_clamped,
                lon=lon_clamped,
                method='nearest'
            )
            
            u = float(pt[u_name].values)
            v = float(pt[v_name].values)
            
            # If nearest gives NaN, try linear
            if np.isnan(u) or np.isnan(v):
                pt = ds_single.interp(
                    lat=lat_clamped,
                    lon=lon_clamped,
                    method='linear'
                )
                u = float(pt[u_name].values)
                v = float(pt[v_name].values)
            
            if np.isnan(u) or np.isnan(v):
                current_speed = np.nan
            else:
                current_speed = np.sqrt(u**2 + v**2)
            
            # Check if point was clamped
            was_clamped = (lat != lat_clamped) or (lon != lon_clamped)
            
        except Exception as e:
            u, v, current_speed = np.nan, np.nan, np.nan
            was_clamped = True
        
        result = {
            'river_name': river_row['river_name'],
            'country': river_row['country'],
            'original_lat': river_row['mouth_lat_deg'],
            'original_lon': river_row['mouth_lon_deg'],
            'adjusted_lon': river_row['lon_adjusted'],
            'used_lat': lat_clamped,
            'used_lon': lon_clamped,
            'date': file_date,
            'u_ms': u,
            'v_ms': v,
            'current_speed_ms': current_speed,
            'was_clamped': was_clamped,
            'file_source': hycom_file.name,
            'depth_type': depth_type
        }
        
        daily_results.append(result)
    
    # Convert daily results to DataFrame
    daily_df = pd.DataFrame(daily_results)
    
    # Calculate statistics for this day
    valid_speeds = daily_df['current_speed_ms'].dropna()
    if len(valid_speeds) > 0:
        print(f"  Valid currents: {len(valid_speeds)}/{len(daily_df)} rivers")
        print(f"  Mean speed: {valid_speeds.mean():.4f} m/s")
        print(f"  Max speed: {valid_speeds.max():.4f} m/s")
        
        # Show top rivers for this day
        top_rivers = daily_df.nlargest(5, 'current_speed_ms')
        print("  Top rivers today:")
        for _, row in top_rivers.iterrows():
            print(f"    {row['river_name']}: {row['current_speed_ms']:.4f} m/s")
    else:
        print(f"  No valid current data found for this day")
    
    all_results.append(daily_df)
    
    # Close dataset
    ds.close()

# ---------- Combine all results ----------
if all_results:
    combined_results = pd.concat(all_results, ignore_index=True)
    
    print(f"\n{'='*60}")
    print(f"COMBINED RESULTS SUMMARY")
    print(f"{'='*60}")
    print(f"Total measurements: {len(combined_results)}")
    print(f"Unique rivers: {combined_results['river_name'].nunique()}")
    print(f"Unique dates: {combined_results['date'].nunique()}")
    
    # Calculate river-wise statistics
    river_stats = combined_results.groupby('river_name').agg({
        'current_speed_ms': ['count', 'mean', 'std', 'min', 'max'],
        'country': 'first',
        'original_lat': 'first',
        'original_lon': 'first'
    }).round(4)
    
    river_stats.columns = ['_'.join(col).strip() for col in river_stats.columns.values]
    river_stats = river_stats.rename(columns={
        'current_speed_ms_count': 'n_days',
        'current_speed_ms_mean': 'avg_speed_ms',
        'current_speed_ms_std': 'std_speed_ms',
        'current_speed_ms_min': 'min_speed_ms',
        'current_speed_ms_max': 'max_speed_ms',
        'country_first': 'country',
        'original_lat_first': 'latitude',
        'original_lon_first': 'longitude'
    })
    
    # Filter rivers with at least some valid data
    rivers_with_data = river_stats[river_stats['n_days'] > 0].copy()
    print(f"\nRivers with valid data: {len(rivers_with_data)}")
    
    if len(rivers_with_data) > 0:
        print("\nTop 10 rivers by average current speed:")
        top_rivers = rivers_with_data.nlargest(10, 'avg_speed_ms')
        print(top_rivers[['country', 'avg_speed_ms', 'max_speed_ms', 'n_days']])
        
        print(f"\nOverall statistics:")
        print(f"  Mean current speed: {rivers_with_data['avg_speed_ms'].mean():.4f} ± {rivers_with_data['avg_speed_ms'].std():.4f} m/s")
        print(f"  Maximum average speed: {rivers_with_data['avg_speed_ms'].max():.4f} m/s")
        print(f"  Minimum average speed: {rivers_with_data['avg_speed_ms'].min():.4f} m/s")
        
        # ---------- Save results ----------
        # Save detailed results
        detailed_path = PROC_PATH / "river_currents_detailed_2024.csv"
        combined_results.to_csv(detailed_path, index=False)
        print(f"\nSaved detailed results to: {detailed_path}")
        
        # Save river summary
        summary_path = PROC_PATH / "river_currents_summary_2024.csv"
        rivers_with_data.to_csv(summary_path)
        print(f"Saved river summary to: {summary_path}")
        
        # Save a simple version with just the essentials
        simple_results = combined_results[['river_name', 'country', 'date', 'current_speed_ms', 'u_ms', 'v_ms']].copy()
        simple_path = PROC_PATH / "river_currents_simple_2024.csv"
        simple_results.to_csv(simple_path, index=False)
        print(f"Saved simple results to: {simple_path}")
        
        # ---------- Create summary report ----------
        report_path = PROC_PATH / "currents_analysis_report.txt"
        with open(report_path, 'w') as f:
            f.write("River Currents Analysis Report\n")
            f.write("=" * 60 + "\n\n")
            f.write(f"Analysis Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
            f.write(f"HYCOM Files Processed: {len(hycom_files)}\n")
            f.write(f"Files: {', '.join([f.name for f in hycom_files])}\n\n")
            f.write(f"HYCOM Region: Lat [{hycom_lat_min:.1f}°, {hycom_lat_max:.1f}°], "
                   f"Lon [{hycom_lon_min:.1f}°, {hycom_lon_max:.1f}°]\n")
            f.write(f"Rivers Analyzed: {len(rivers_in_region)}\n")
            f.write(f"Rivers with Valid Data: {len(rivers_with_data)}\n\n")
            
            f.write("Overall Statistics:\n")
            f.write(f"  Mean Current Speed: {rivers_with_data['avg_speed_ms'].mean():.4f} m/s\n")
            f.write(f"  Std Dev: {rivers_with_data['avg_speed_ms'].std():.4f} m/s\n")
            f.write(f"  Range: {rivers_with_data['avg_speed_ms'].min():.4f} to {rivers_with_data['avg_speed_ms'].max():.4f} m/s\n\n")
            
            f.write("Top 10 Rivers by Current Speed:\n")
            for idx, (river_name, row) in enumerate(top_rivers.iterrows(), 1):
                f.write(f"{idx:2d}. {river_name:30s} {row['country']:20s} "
                       f"Avg: {row['avg_speed_ms']:.4f} m/s  Max: {row['max_speed_ms']:.4f} m/s  "
                       f"Days: {row['n_days']}\n")
            
            f.write(f"\nFiles Generated:\n")
            f.write(f"  1. {detailed_path.name} - Detailed daily measurements\n")
            f.write(f"  2. {summary_path.name} - River-wise statistics\n")
            f.write(f"  3. {simple_path.name} - Simplified daily data\n")
            f.write(f"  4. {report_path.name} - This report\n")
        
        print(f"\nSaved analysis report to: {report_path}")
        
        # ---------- Create visualization ----------
        try:
            import matplotlib.pyplot as plt
            
            # Create time series plot for top rivers
            top_river_names = top_rivers.index.tolist()[:5]
            
            plt.figure(figsize=(14, 8))
            
            for river_name in top_river_names:
                river_data = combined_results[combined_results['river_name'] == river_name]
                river_data = river_data.sort_values('date')
                
                plt.plot(river_data['date'], river_data['current_speed_ms'], 
                        marker='o', linewidth=2, markersize=5, label=river_name)
            
            plt.title('Daily Current Speeds for Top Rivers (August-September 2024)', fontsize=14)
            plt.xlabel('Date', fontsize=12)
            plt.ylabel('Current Speed (m/s)', fontsize=12)
            plt.grid(True, alpha=0.3)
            plt.legend()
            plt.xticks(rotation=45)
            plt.tight_layout()
            
            time_series_path = PROC_PATH / "currents_time_series.png"
            plt.savefig(time_series_path, dpi=150, bbox_inches='tight')
            plt.close()
            
            print(f"Saved time series plot to: {time_series_path}")
            
        except ImportError:
            print("\nNote: Install matplotlib for visualizations: pip install matplotlib")
        
    else:
        print("\nWARNING: No valid current data found in any files!")
        print("Possible issues:")
        print("1. River coordinates may be incorrect")
        print("2. HYCOM data may have different coordinate system")
        print("3. Rivers may be on land where ocean currents aren't defined")
        
        # Still save the empty results for debugging
        debug_path = PROC_PATH / "river_currents_debug.csv"
        combined_results.to_csv(debug_path, index=False)
        print(f"\nSaved debug data to: {debug_path}")
        
else:
    print("\nERROR: No results were generated!")

print(f"\n{'='*60}")
print("PROCESSING COMPLETE")
print(f"{'='*60}")

RAW_PATH: C:\Users\s572269\Downloads\river_plastic_predictor\data\raw
PROC_PATH: C:\Users\s572269\Downloads\river_plastic_predictor\data\processed
Total rivers: 1387

Analyzing river coordinate ranges:
Min/Max mouth_lat: -6699114.72, 13241189.53
Min/Max mouth_lon: -18310017.91, 19761475.71

After scaling by 100,000:
    river_name  mouth_lat_deg  mouth_lon_deg
0       Rungwa      -8.038435      35.622237
1      Ligonha     -17.036085      41.608649
2       Dongwe     -15.596923      26.644385
3        Cuito     -14.198700      20.414734
4        Bagoé      12.960210      -7.303820
5      Hadejia      12.849537       8.341243
6         Sous      35.505781      -9.517816
7  Oum Er Rbia      38.288080      -6.679169
8   San Miguel     -14.993571     -71.244474
9        Coari      -4.465621     -70.328643

Lat range: -66.99 to 132.41
Lon range: -183.10 to 197.61

Rivers after cleaning: 1387

Found 6 HYCOM files:
  uv3z_08-31-2024.nc4
  uv3z_09-01-2024.nc4
  uv3z_09-02-2024.nc4
  uv3z_09-03